In [ ]:
!pip install opencv-python

# db 에서 불러오기

In [1]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

# db connect 함수

def db_connection(host_name='ds'): 
    
    host_url = "db.ds.mycelebs.com"
    user_nm = "dbteam"
    passwd = "celebdbteam2@#"
    port_num = 3306
    db_name = "sephora_keycolor"
    
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8', 
                           db = db_name,cursorclass=pymysql.cursors.DictCursor) 
    # cursorclass=pymysql.cursors.DictCursor 추가 -> 데이터프레임으로 다루기 쉽게 딕셔너리 형태로 결과 반환해줌, cursor는 튜플형태  
    # db=db,
    
    return conn

sql = '''

select * from sephora_keycolor.download_image_info where resource = 'pinterest'

'''
cursor = db_connection().cursor(pymysql.cursors.DictCursor)
cursor.execute(sql)

result = cursor.fetchall()

# fetchall() : 모든 데이터를 한 번에 가져옴
# fetchone() : 한 번 호출에 하나의행만 가져옴
# fetchmany(n) : n개 만큼의 데이터를 가져옴

# 결과를 pandas 데이터 프레임으로 변환


data = pd.DataFrame(result)


In [19]:
df_61 = data[data['category_uid'] == 61]
df_64 = data[data['category_uid'] == 64]
df_64 = df_64.reset_index().drop(columns = 'index', axis=1)
df= df_64[:100]

In [17]:
df


,pk,search_query,kl_name,category_uid,image_url,image_path,is_use,is_main,resource
0,288988,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/9c/88/62/9c8862666a2...,200825/64/absolutely_stunning/img_1_13225_abso...,1,1,pinterest
1,288989,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/14/05/ae/1405ae9d5ed...,200825/64/absolutely_stunning/img_2_13225_abso...,1,1,pinterest
2,288990,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/e6/5b/eb/e65beb9eb5e...,200825/64/absolutely_stunning/img_3_13225_abso...,1,1,pinterest
3,288991,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/ce/7b/4e/ce7b4e36100...,200825/64/absolutely_stunning/img_4_13225_abso...,1,1,pinterest
4,288992,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/ef/58/cc/ef58cc00217...,200825/64/absolutely_stunning/img_5_13225_abso...,1,1,pinterest
...,...,...,...,...,...,...,...,...,...
95,289083,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/95/20/59/95205943126...,200825/64/absolutely_stunning/img_96_13225_abs...,1,1,pinterest
96,289084,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/2b/3f/1b/2b3f1bbec11...,200825/64/absolutely_stunning/img_97_13225_abs...,1,1,pinterest
97,289085,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/9f/d1/70/9fd170bb967...,200825/64/absolutely_stunning/img_98_13225_abs...,1,1,pinterest
98,289086,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/d3/e2/e3/d3e2e323829...,200825/64/absolutely_stunning/img_99_13225_abs...,1,1,pinterest


# MAIN

In [20]:
import cv2
from tqdm import tqdm
import os

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
df['image_path2'] = ""
for i in tqdm(range(len(df))):
    try:
        image = cv2.imread(df['image_path'][i])
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.05, minNeighbors = 2) 
        
       
        # 1 또는 -1 로 분류하기
        if len(faces) >= 1:
            df['is_use'][i] = 1
        else:
            df['is_use'][i] = -1
        

    except:
        df['is_use'][i] = 0

    
df



<ipython-input-20-89c1857c2c69>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_path2'] = ""
  0%|          | 0/100 [00:00<?, ?it/s]<ipython-input-20-89c1857c2c69>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_use'][i] = 1
/Users/choiswonspec/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

,pk,search_query,kl_name,category_uid,image_url,image_path,is_use,is_main,resource,image_path2
0,288988,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/9c/88/62/9c8862666a2...,200825/64/absolutely_stunning/img_1_13225_abso...,1,1,pinterest,
1,288989,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/14/05/ae/1405ae9d5ed...,200825/64/absolutely_stunning/img_2_13225_abso...,1,1,pinterest,
2,288990,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/e6/5b/eb/e65beb9eb5e...,200825/64/absolutely_stunning/img_3_13225_abso...,1,1,pinterest,
3,288991,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/ce/7b/4e/ce7b4e36100...,200825/64/absolutely_stunning/img_4_13225_abso...,1,1,pinterest,
4,288992,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/ef/58/cc/ef58cc00217...,200825/64/absolutely_stunning/img_5_13225_abso...,1,1,pinterest,
...,...,...,...,...,...,...,...,...,...,...
95,289083,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/95/20/59/95205943126...,200825/64/absolutely_stunning/img_96_13225_abs...,1,1,pinterest,
96,289084,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/2b/3f/1b/2b3f1bbec11...,200825/64/absolutely_stunning/img_97_13225_abs...,1,1,pinterest,
97,289085,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/9f/d1/70/9fd170bb967...,200825/64/absolutely_stunning/img_98_13225_abs...,1,1,pinterest,
98,289086,absolutely_stunning_makeup,absolutely_stunning,64,https://i.pinimg.com/564x/d3/e2/e3/d3e2e323829...,200825/64/absolutely_stunning/img_99_13225_abs...,1,1,pinterest,


## 함수로 apply

In [18]:

from tqdm import tqdm
tqdm.pandas()

def openCV(path):
    try:
        image = cv2.imread(path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.05, minNeighbors = 2) 
        # 1 또는 -1 로 분류하기
        if len(faces) >= 1:
            return 1
        else:
            return -1
    except:
        return 0
    
df['is_use'] = df['image_path'].progress_apply(openCV)

/Users/choiswonspec/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 100/100 [00:18<00:00,  5.26it/s]
<ipython-input-18-66ccf4feb8f4>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_use'] = df['image_path'].progress_apply(openCV)


In [21]:
df['is_use'].value_counts()

 1    89
-1    11
Name: is_use, dtype: int64

#### 이미지 다시 저장하는법

shutil

In [ ]:



import glob

path = "/Users/choiswonspec/PINTEREST/200825/64/*"
file_list = glob.glob(path)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

for file in file_list:
    path2 = file + "/" + "*"
    image_list = glob.glob(path2)
    for image in image_list:
        n= cv2.imread(image)
        gray = cv2.cvtColor(n, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.05, minNeighbors = 2) 
        
        
        # 파일 이름 변경
        gijun = path.split("/")
    
        if gijun[1] == '61':
            gijun[1] = df['search_query'][i]
        elif gijun[1] == '62':
            gijun[1] = df['search_query'][i]
        elif gijun[1] == '64':
            gijun[1] = df['search_query'][i]
        else:
            gijun[1] = df['search_query'][i]

        changed_path_name = "/".join(gijun)
    
            

    
#file_list_py = [file for file in file_list if file.endswith(".jpg")]
file_list
#print ("file_list_py: {}".format(file_list_py))
# 지정된 경로에서 .jpg 이미지 파일을 모두 불러오기



# images 변수에 등록된 파일을 하나씩 불러 데이터 처리

#images

In [ ]:
import cv2
# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Read the input image 이미지 불러오기
img = cv2.imread('./img_2_21_absolutely_adorable.jpg')
# img = cv2.imread('./img_9_21_absolutely_adorable.jpg')

# 이미지는 꼭 grayscale 이어야 하기 때문에 전환해준다
# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 5) 
eyes = eye_cascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 5)
# 이 떄, len(faces) 는 얼굴의 갯수가 출력됨....


# # Draw rectangle around the faces 시각적으로 표현해주기 위해 그리는 작업.....
# for (x, y, w, h) in faces:
#     cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
# Draw rectangle around the eyes
for (x, y, w, h) in eyes:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
# Display the output
cv2.imshow('img', img)
cv2.waitKey()
len(faces)